#   **QUANTUM KEY DISTRIBUTION & QUANTUM CRYPTOGRAPHY**

#1.Introduction

Quantum cryptography is the study of quantum mechanic properties to perform cryptographic tasks. The prominent feature of quantum cyptography is the fact that it enables the completion of cryptographic tasks that are proven to be impossible to be carried out on classical communication. The following are some examples of Quantum cryptography.
*   Quantum Key Distribution
*   Mistrustful Quantum Crptography
*   Quantum Coin Flipping
*   Quantum Commitment
This explainer, in particular is foucused on one of the quantum key distribution protocol known as BB84 named after its developers Charles Bennet and Gillles Brassard, both of whom are also known for Quantum Teleportation. 

Quantum key distribution, is the best-known example of a quantum cryptographic task. It allows two parties to produce a shared random secret key known only to them. Moreover, it also has a unique ability to detect any third party trying to gain knowledge of the key. This is a result from a fundamental quantum mechanics : the process of measuring a quantum system in general disturbs the system. But a third party trying to eavesdrop on the key must measure it, producing detectable anomalies. With the help of quantum superpostions or entanglement and transmitting information in quantum states, a communication system can be implemented that detects eavesdropping. If the level of eavesdropping is below a certain threshold, a key can be produced that is guranteed to be secure, otherwise no secure key is possible and communication is aborted must be restarted.

#2.BB84 Protocol Overview

As mentioned above, BB84, one of the quantum key distribution protocol was developed by Charles Benneet and Gilles Brassard, both of whom are also known for Quantum Teleportation. This protocol was also the first QKD protocol proposed. This protocol is widely known as a method of securely communicating a private key from one party to another for use in one-time pad encryption.

This protocol starts with two parties Alice and Bob respectively connected by a classical communication channel. Alice wishes to send Bob a private key. She starts of with two strings of bits, $a$ and $b$, with the same length of $n$. Following this she encodes them in accordance to the following desctiption:

*   $|\psi_{00}\rangle$ =$|0\rangle$
*   $|\psi_{01}\rangle$ =$|1\rangle$
*   $|\psi_{10}\rangle$ =$|+\rangle$
*   $|\psi_{11}\rangle$ =$|-\rangle$





Then, Alice send her qubits to Bob, who then generatees a random binary string of $c$, with the same length of $n$. He declares the receiving of the qubits and measures each qubit in the $X$ or $Z$ basis at random and stored it as $m$.

Then, they both reveal the strings $b$ and $c$ respectively. If, $b$ and $c$ are the same, $a$ and $m$ matches in those places. This occured as the qubit was measured using the same basis in which it was encoded. Alice and Bob discard any bits where Bob measured a different basis than Alice prepared.
The bits from strings $a$ and $m$ where the bases match can be used as their secured key.

Accroding to the no-cloning theorem, a qubit that is an unknownn state at the start cannot be copied of cloned. Hence, any measurement will destroy the initial state of the qubit. This will also illustrated below in the implementation. But in general, suppose there exits a third party named Eve. She intercepts all of Alice's qubits, measures them in a randomly chosen basis and resends it back to Bob. The state Eve measures is not necessariy the state Alice prepared. Thus, Alice and Bob will not end up having the same out come for that qubit even if their basis choices match. With the help of this, Bob and Alice can now detect eavesdropping and abort the protocol.





#3.Simulation of Non-eavesdropped protocol

In [ ]:
# install cirq
!pip install cirq --quiet

     |████████████████████████████████| 1.4MB 2.9MB/s 


In [ ]:
import cirq
import numpy as np

Alice generates her random set of bits and choose to encode each of qubit in the $X$ and $Z$ basis at random, and stores the choice for each qubit in ```alice_bases```. 0 means prepared in $Z$ and 1 means prepared in $X$.



In [ ]:
n = 10
# Alice's random bits
alice_bits = [np.random.randint(0,2) for _ in range(n)]
print ('Bits generated by Alice,' )
print(alice_bits)

# Alice's random bases for each bits
alice_bases = [np.random.randint(0,2) for _ in range(n)]
print ('Encoded bases of each qubits of Alice,' )
print(alice_bases)


NameError: ignored

Now, Alice must encode the message and send it to bob and bob needs to measure the encoded message. And the following fucntion allows Alice to encode the qubits

In [ ]:
def encode_measure(n,alice_bits,alice_bases): #Encoding
  qubits = [cirq.LineQubit(i) for i in range (n)]
  circuit = cirq.Circuit()
  alice_encoding = []
  for i,  _ in enumerate(alice_bases):
    if alice_bits[i] == 1:
      alice_encoding.append(cirq.X(qubits[i]))
    if alice_bases[i] == 1:
      alice_encoding.append(cirq.H(qubits[i]))
  circuit.append(alice_encoding)
  return circuit


As shown below, ```encode_measure``` generates a list of QuantumCircuits, each representing a single qubit in Alice's message


In [ ]:
n = 10
# Alice's random bits
alice_bits = [np.random.randint(0,2) for _ in range(n)]

# Alice's random bases for each bits
alice_bases = [np.random.randint(0,2) for _ in range(n)]
encoded = encode_measure(n,alice_bits,alice_bases)
print(alice_bases)
print(encoded)

The encoded message is then passed to Bob

In [ ]:
n = 10
# Alice's random bits
alice_bits = [np.random.randint(0,2) for _ in range(n)]

# Alice's random bases for each bits
alice_bases = [np.random.randint(0,2) for _ in range(n)]
encoded = encode_measure(n,alice_bits,alice_bases)

#Bob bases to be used to measure the message
bob_bases = [np.random.randint(0,2) for _ in range(n)]
print(bob_bases)

With the help of the fucntion below Bob measures each qubit in the message in $X$ or $Z$ basis at random and stores the information.

In [ ]:
def measure(message,bob_bases):
  qubits = [cirq.LineQubit(i) for i in range(n)]
  circuit = cirq.Circuit()
  
  for i, _ in enumerate(bob_bases):
    if bob_bases[i] == 1:
     message.append(cirq.H(qubits[i]))
  circuit.append(message)
  circuit.append(cirq.measure_each(*qubits))
  return circuit


In [ ]:
n = 10
# Alice's random bits
alice_bits = [np.random.randint(0,2) for _ in range(n)]

# Alice's random bases for each bits
alice_bases = [np.random.randint(0,2) for _ in range(n)]
encoded = encode_measure(n,alice_bits,alice_bases)
print(encoded)

#Bob bases to be use to measure the message
bob_bases = [np.random.randint(0,2) for _ in range(n)]
measured = measure(encoded,bob_bases)
print(bob_bases)
print(measured)

In [ ]:
def bitstring(bits):
  return ''.join(str(int(b)) for b in bits)

Then, Bob and Alice reveals their bases and ```generate_expectedkey``` function will generate bits which they can use as their key and discard the useless bits.

In [ ]:
def generate_expectedkey(alice_bases,bob_bases):
  key = bitstring([
                          alice_bits[i]
                         for i in range(n)
                          if alice_bases[i] == bob_bases[i]
])
  return key

In [ ]:
n = 10

# Alice's random bits
alice_bits = [np.random.randint(0,2) for _ in range(n)]

# Alice's random bases for each bits
alice_bases = [np.random.randint(0,2) for _ in range(n)]

# Alice encoded qubits
encoded = encode_measure(n,alice_bits,alice_bases)

#Bob bases to be use to measure the message
bob_bases = [np.random.randint(0,2) for _ in range(n)]
measured = measure(encoded,bob_bases)

#Expected Key after sharing bases
expected_key = generate_expectedkey(alice_bases,bob_bases)
print(expected_key)

In [ ]:
n = 10
# Alice's random bits
alice_bits = [np.random.randint(0,2) for _ in range(n)]

# Alice's random bases for each bits
alice_bases = [np.random.randint(0,2) for _ in range(n)]

# Alice encoded qubits
encoded = encode_measure(n,alice_bits,alice_bases)

#Bob bases to be use to measure the message
bob_bases = [np.random.randint(0,2) for _ in range(n)]
measured = measure(encoded,bob_bases)

#Expected Key after sharing bases
expected_key = generate_expectedkey(alice_bases,bob_bases)

#Simulate
repetitions = 1
result = cirq.Simulator().run(measured,repetitions=repetitions)
result_bitstring = bitstring([int(result.measurements[str(i)]) for i in range(n)])
print(result_bitstring)

In [ ]:
def generate_obtainedkey(ressult_bitstring,bob_bases,alice_bases):
    key = bitstring([
                     result_bitstring[i]
                     for i in range(n)
                       if alice_bases[i]==bob_bases[i]

    ])
    return key

In [ ]:
n = 10
# Alice's random bits
alice_bits = [np.random.randint(0,2) for _ in range(n)]

# Alice's random bases for each bits
alice_bases = [np.random.randint(0,2) for _ in range(n)]

# Alice encoded qubits
encoded = encode_measure(n,alice_bits,alice_bases)

#Bob bases to be use to measure the message
bob_bases = [np.random.randint(0,2) for _ in range(n)]
measured = measure(encoded,bob_bases)

#Expected Key after sharing bases
expected_key = generate_expectedkey(alice_bases,bob_bases)
print(expected_key)

#Simulate
repetitions = 1
result = cirq.Simulator().run(measured,repetitions=repetitions)
result_bitstring = bitstring([int(result.measurements[str(i)]) for i in range(n)])

#Actual Key after Simulation
obtained_key = generate_obtainedkey(result_bitstring,bob_bases,alice_bases)
print(obtained_key)

Finally, Bob and Alice compare a random selectin of the bits in their keys to ensure that the protocal works smoothly.

In [ ]:

n = 10
# Alice's random bits
alice_bits = [np.random.randint(0,2) for _ in range(n)]

# Alice's random bases for each bits
alice_bases = [np.random.randint(0,2) for _ in range(n)]

# Alice encoded qubits
encoded = encode_measure(n,alice_bits,alice_bases)

#Bob bases to be use to measure the message
bob_bases = [np.random.randint(0,2) for _ in range(n)]
measured = measure(encoded,bob_bases)

#Expected Key after sharing bases
expected_key = generate_expectedkey(alice_bases,bob_bases)

#Simulate
repetitions = 1
result = cirq.Simulator().run(measured,repetitions=repetitions)
result_bitstring = bitstring([int(result.measurements[str(i)]) for i in range(n)])

#Actual Key after Simulation
obtained_key = generate_obtainedkey(result_bitstring,bob_bases,alice_bases)

#Print the Protocol
print('Simulation without Interception(Non-Eavesdropped)')
print('')
print(measured)
print('')
print_results(alice_bases, bob_bases, alice_bits, expected_key,obtained_key)

The below function is used for printing protocols

In [ ]:
def print_results(alice_bases, bob_bases, alice_bits, expected_key,
                  obtained_key):
    n = len(alice_bases)
    basis_match = ''.join([
        'O' if alice_bases[i] == bob_bases[i] else '-'
        for i in range(n)
    ])
    alice_bases_string = "".join(
        ['C' if alice_bases[i] == 0 else "H" for i in range(n)])
    bob_bases_string = "".join(
        ['C' if bob_bases[i] == 0 else "H" for i in range(n)])

    print('Alice\'s basis:\t{}'.format(alice_bases_string))
    print('Bob\'s basis:\t{}'.format(bob_bases_string))
    print('Alice\'s bits:\t{}'.format(bitstring(alice_bits)))
    print('Bases match::\t{}'.format(basis_match))
    print('Expected key:\t{}'.format(expected_key))
    print('Actual key:\t{}'.format(obtained_key))

#4.Simulation with Eavesdropped protocol

As for the simulation with interception, we repeat the same step as above, but before the message arrives to Bob, Eve will try to inercept and extract information from the message.

In [ ]:
np.random.seed(seed=3)
n=10
# Alice's random bits
alice_bits = [np.random.randint(0,2) for _ in range(n)]

# Alice's random bases for each bits
alice_bases = [np.random.randint(0,2) for _ in range(n)]

# Alice encoded qubits
encoded = encode_measure(n,alice_bits,alice_bases)
print(encoded)



And hence the interception begins here. Eve intecepted the message and try to measure it with her own random selection of bases.

In [ ]:
np.random.seed(seed=3)
n = 10
# Alice's random bits
alice_bits = [np.random.randint(0,2) for _ in range(n)]

# Alice's random bases for each bits
alice_bases = [np.random.randint(0,2) for _ in range(n)]

# Alice encoded qubits
encoded = encode_measure(n,alice_bits,alice_bases)

# Interception!!!!
eves_bases = [np.random.randint(0,2) for _ in range(n)]
intercepted_message = measure(encoded, eves_bases)
print(intercepted_message)


0: ───M───────────────

1: ───H───H───M───────

2: ───X───H───H───M───

3: ───X───H───M───────

4: ───M───────────────

5: ───H───M───────────

6: ───H───H───M───────

7: ───X───M───────────

8: ───X───M───────────

9: ───X───M───────────


The simulation below generates the bit string of Eve's measurement

In [ ]:
np.random.seed(seed=3)
n = 10
# Alice's random bits
alice_bits = [np.random.randint(0,2) for _ in range(n)]

# Alice's random bases for each bits
alice_bases = [np.random.randint(0,2) for _ in range(n)]

# Alice encoded qubits
encoded = encode_measure(n,alice_bits,alice_bases)

# Interception!!!!
eves_bases = [np.random.randint(0,2) for _ in range(n)]
intercepted_message = measure(encoded, eves_bases)

#Simulate
repetitions = 1
result = cirq.Simulator().run(intercepted_message,repetitions=repetitions)
eaves_bitstring = bitstring([int(result.measurements[str(i)]) for i in range(n)])
print(eaves_bitstring)



Finally, Bob have received the intercepted message. And by looking at the output circuits, one can see that the qubits states have been altered. And hence no-cloning theoem plays a huge role in this protocol. But for now, neither Bob nor Alice has noticed the interception.

In [ ]:
np.random.seed(seed=3)
n = 10
# Alice's random bits
alice_bits = [np.random.randint(0,2) for _ in range(n)]

# Alice's random bases for each bits
alice_bases = [np.random.randint(0,2) for _ in range(n)]

# Alice encoded qubits
encoded = encode_measure(n,alice_bits,alice_bases)
print('Alice')
print(encoded)

# Interception!!!!
eves_bases = [np.random.randint(0,2) for _ in range(n)]
intercepted_message = measure(encoded, eves_bases)

#Simulate
repetitions = 1
result = cirq.Simulator().run(intercepted_message,repetitions=repetitions)
eaves_bitstring = bitstring([int(result.measurements[str(i)]) for i in range(n)])

#Bob received the intercepted message and measure it with his own bases 
bob_bases = [np.random.randint(0,2) for _ in range(n)]
measured = measure(encoded,bob_bases)
print('Eve')
print(eves_bases)
print(intercepted_message)
print('Bob')
print(bob_bases)
print(measured)

Alice
1: ───H───────

2: ───X───H───

3: ───X───H───

5: ───H───────

6: ───H───────

7: ───X───────

8: ───X───────

9: ───X───────
Eve
[0, 1, 1, 0, 0, 0, 1, 0, 0, 0]
0: ───M───────────────

1: ───H───H───M───────

2: ───X───H───H───M───

3: ───X───H───M───────

4: ───M───────────────

5: ───H───M───────────

6: ───H───H───M───────

7: ───X───M───────────

8: ───X───M───────────

9: ───X───M───────────
Bob
[0, 1, 1, 0, 1, 0, 0, 1, 1, 0]
0: ───M───────────────────

1: ───H───H───H───M───────

2: ───X───H───H───H───M───

3: ───X───H───M───────────

4: ───H───M───────────────

5: ───H───M───────────────

6: ───H───H───M───────────

7: ───X───H───M───────────

8: ───X───H───M───────────

9: ───X───M───────────────


Bob and Alice reveal their basis choice and then discard their useless bit. Then, they compare the same random selection of their keys to see if the quibits were intercepted. And if their keys do not match, they will now realized that the message has been intercepted and altered and must abort the protocol altogether. Eve's attempt to intercept has failed.

In [ ]:
np.random.seed(seed=1)
n = 10
# Alice's random bits
alice_bits = [np.random.randint(0,2) for _ in range(n)]

# Alice's random bases for each bits
alice_bases = [np.random.randint(0,2) for _ in range(n)]

# Alice encoded qubits
encoded = encode_measure(n,alice_bits,alice_bases)

# Interception!!!!
eves_bases = [np.random.randint(0,2) for _ in range(n)]
intercepted_message = measure(encoded, eves_bases)

#Simulate
repetitions = 1
result = cirq.Simulator().run(intercepted_message,repetitions=repetitions)
eaves_bitstring = bitstring([int(result.measurements[str(i)]) for i in range(n)])

#Bob received the intercepted message and measure it with his own bases 
bob_bases = [np.random.randint(0,2) for _ in range(n)]
measured = measure(encoded,bob_bases)

#Expected Key after sharing bases
expected_key = generate_expectedkey(alice_bases,bob_bases)

#Simulate
repetitions = 1
result = cirq.Simulator().run(measured,repetitions=repetitions)
result_bitstring = bitstring([int(result.measurements[str(i)]) for i in range(n)])

#Actual Key after Simulation
obtained_key = generate_obtainedkey(result_bitstring,bob_bases,alice_bases)

#Print the Protocol
print('Simulation with Interception(Eavesdropped)')
print('')
print(measured)
print('')
print_results(alice_bases, bob_bases, alice_bits, expected_key,obtained_key)
if expected_key != obtained_key:
  print("Eve's inteference was detected")
else:
  print('Eve went un-detected!')

Simulation with Interception(Eavesdropped)

0: ───X───H───M───────────

1: ───X───H───H───H───M───

2: ───H───M───────────────

3: ───H───H───M───────────

4: ───X───H───H───H───M───

5: ───X───M───────────────

6: ───X───M───────────────

7: ───X───H───M───────────

8: ───X───H───H───M───────

9: ───H───M───────────────

Alice's basis:	CHCHHCCHCC
Bob's basis:	HHHHHCCCHH
Alice's bits:	1100111110
Bases match::	-O-OOOO---
Expected key:	10111
Actual key:	00111
Eve's inteference was detected


For this type of interception, there is a narrow window of chance that Bob and Alice's samples could match and Alice sends her vulnerable message through Eve's channel. In this case Eve's interception attempt was not detected.